In [12]:
import pandas as pd
import os
import modules.graph as graph
import modules.geo as geo

In [13]:
data_file = 'UT_profits.xlsx'
df_profits = pd.read_excel(data_file)

data_file = 'Colegios2020(UTs).xlsx'
df_schools = pd.read_excel(data_file)

In [14]:
uts = df_profits['UT'].unique()
UT = [df_schools['UT'][i] for i in range(len(df_schools))]

ut_edges = [[] for _ in range(len(uts))]

points = [(df_schools['Longitud'][i], df_schools['Latitud'][i]) 
          for i in range(len(df_schools))]

schools_edges = graph.get_adj_list(points)

for i in range(len(df_schools)):
    for j in schools_edges[i]:

        dist = geo.convert_to_meters(graph.distance(points[i], points[j]))

        if UT[j] not in ut_edges[UT[i]] and UT[i] != UT[j] and dist < 150000:
            ut_edges[UT[i]].append(UT[j])

In [15]:
sumdiff = 0
maxdiff = 0
mindiff = 100000
contar = 0

profit = [df_profits['Profit'][i] for i in range(len(df_profits))]

for i in range(len(ut_edges)):
    for j in ut_edges[i]:
        sumdiff = sumdiff + max(profit[i], profit[j]) / min(profit[i], profit[j])
        contar = contar + 1
        maxdiff = max(maxdiff, max(profit[i], profit[j]) / min(profit[i], profit[j]))
        mindiff = min(mindiff, max(profit[i], profit[j]) / min(profit[i], profit[j]))

print(sumdiff/contar)
print(maxdiff)
print(mindiff)

1.2804464629206127
2.1387714268300004
1.0011206007660922
